# Extract and Clean Data from all_tournaments.csv

Primero creamos un Spark Context

In [1]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
from pyspark.sql.types import  (StructType, StructField, DateType, BooleanType, DoubleType, IntegerType, StringType, TimestampType)
from pyspark.sql.functions import col, udf
import os

spark = SparkSession.builder.master("local[1]").appName("tenis-matches").getOrCreate()

In [19]:
raw_file_path = os.path.join(os.path.abspath(os.path.pardir), "dataset", "raw", "all_tournaments.csv")
tor = spark.read.csv("file:///" + raw_file_path, header = True)

C:\Users\fedsa\Documents\ORT\Semestre 9\Big Data\Obligatorio\dataset\raw\all_tournaments.csv


In [20]:
print("There are {} tournaments.".format(tor.count()))

There are 36488 tournaments.


### Eliminaremos algunas columnas del dataset que no nos sirven

In [6]:
tor.printSchema()

root
 |-- year: string (nullable = true)
 |-- tournament: string (nullable = true)
 |-- start_date: string (nullable = true)
 |-- end_date: string (nullable = true)
 |-- location: string (nullable = true)
 |-- court_surface: string (nullable = true)
 |-- prize_money: string (nullable = true)
 |-- currency: string (nullable = true)
 |-- masters: string (nullable = true)



Eliminaremos las siguientes columnas que no serán útiles para nuestro análisis:
- prize_money
- currency
- masters

In [7]:
tor = tor.drop("prize_money").drop("currency").drop("masters")

In [8]:
tor.printSchema()

root
 |-- year: string (nullable = true)
 |-- tournament: string (nullable = true)
 |-- start_date: string (nullable = true)
 |-- end_date: string (nullable = true)
 |-- location: string (nullable = true)
 |-- court_surface: string (nullable = true)



No eliminaremos torneos con valores nulos, ya que puede generar inconsistencias con los otros datasets.

In [25]:
processed_file_path = os.path.join(os.path.abspath(os.path.pardir), "dataset", "processed", "all_tournaments.csv")
tor.write.format("csv").option("header", True).mode('overwrite').save("file:///" + processed_file_path)

## Creacion de archivo para ejecutar extraccion de datos

In [10]:
import os

get_processed_data_script_file = os.path.join(os.path.pardir, "process_all_tournaments.py")

In [32]:
%%writefile $get_processed_data_script_file

import os
import findspark
findspark.init()

from pyspark.sql import SparkSession
from pyspark.sql.types import  (StructType, StructField, DateType, BooleanType, DoubleType, IntegerType, StringType, TimestampType)
from pyspark.sql.functions import col, udf

def extract_data():
    spark = SparkSession.builder.master("local[1]").appName("tenis-matches").getOrCreate()
    
    raw_file_path = os.path.join(os.path.abspath(os.path.pardir), "dataset", "raw", "all_tournaments.csv")
    tor = spark.read.csv("file:///" + raw_file_path, header = True)
    
    tor = tor.drop("prize_money").drop("currency").drop("masters")
    
    return tor
    
if __name__ == '__main__':
    df = extract_data()
    processed_file_path = os.path.join(os.path.abspath(os.path.pardir), "dataset", "processed", "all_tournaments.csv")
    df.write.format("csv").option("header", True).mode('overwrite').save("file:///" + processed_file_path)

Overwriting ..\process_all_tournaments.py


In [33]:
!python $get_processed_data_script_file

SUCCESS: The process with PID 7688 (child process of PID 13808) has been terminated.
SUCCESS: The process with PID 13808 (child process of PID 12904) has been terminated.
SUCCESS: The process with PID 12904 (child process of PID 21124) has been terminated.


The system cannot find the path specified.
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
2020-11-09 22:46:06,065 WARN util.Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
2020-11-09 22:46:06,066 WARN util.Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
2020-11-09 22:46:06,066 WARN util.Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
